# `gpytorch` - Classification

## Overview

In this example we will use a Radial Basis Function (RBF) kernel in a `VariationalGP` module for classification.
The basic model is usable when there is limited training data and no advanced techniques are required.

In this example we're modeling a unit wave with period 1/2 centered with positive values at $x=0$.
We are going to classify the points as either $+1$ or $-1$.

Varitional Inference uses the assumption that the posterior distribution factors multiplicatively over the input variables.
This makes approximating the distribution via the __Kullback-Leibler__ (KL) divergence possible to obtain a fast approximation to the posterior.

For a good explanation of variational techniques, refer to sections 4-6 of [this](https://www.cs.princeton.edu/courses/archive/fall11/cos597C/lectures/variational-inference-i.pdf) document.
In addtion [Variational Inference: A Review for Statisticians](https://arxiv.org/abs/1601.00670) by Blei _et al_, and [Scalable Variational Gaussian Process Classification](https://arxiv.org/abs/1411.2005) by Hensman _et al_ provide a more thorough discussion of these techniques.

In [1]:
%matplotlib inline
%load_ext memory_profiler

In [2]:
import math
import torch
import gpytorch
import matplotlib.pyplot as plt
from tqdm import tnrange

## Training Data

In [3]:
train_x = torch.linspace(0, 1, 10)
train_y = torch.sign(torch.cos(train_x * (4 * math.pi)))

## Defining the Model

Exact inference is intractable for GP classification, so `gpytorch` approximates the classification posterior using __variational inference__.
Variational Inference is ideal in this context for a number of reasons.
First, variational inference commonly relies on gradient descent techniques, which take full advantage of PyTorch's `autograd`.
Additionally, variational inference can be performed with stochastic gradient decent, which is extremely scalable for large datasets.

The basics of defining a `gpytorch` model are covered in the notebook `gpytorch-regression.ipynb`, and a classification model will contain largely the same parts.

1. A __GP Model__ - `gpytorch.models.VariationalGP` - This handles basic variational inference
2. A __Likelihood__ - `gpytorch.likelihoods.BernoulliLikelihood` - This is the ideal likelihood for binary classification
3. A __Mean__ - `gpytorch.means.ConstantMean()` - This defines the prior mean of the GP
4. A __Kernel__ - `gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())` - This defines the prior covariance of the GP
5. A __Multivariate Normal Distribution__ - `gpytorch.distributions.MultivariateNormal` - This is the object used to represent multivariate normal distributions.

> For those unfamiliar with GP classification: even though we are performing classifcation, the GP model still returns a `MultivariateNormal`, but the _likelihood_ transforms this latent Gaussian variable into a Bernoulli variable. 

### The GP Model 

The `VariationalGP` model is `gpytorch`'s simplist approximate inference model.
It approximates the true posterior with a multivariate normal distribution.
The model defines all the variational parameters that are needed.

The necessary components of a `VariationalGP` model are similar to that of the `ExactGP` used in the regression setting.
These components are attributes in the `__init__` method for the mean and the covariance, and a `forward` method for evaluation.

In [4]:
class GPClassificationModel(gpytorch.models.VariationalGP):
    def __init__(self, train_x):
        super(GPClassificationModel, self).__init__(train_x)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        latent_pred = gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
        
        return latent_pred

### Initialize Model and Likelihood

In [5]:
model = GPClassificationModel(train_x)
likelihood = gpytorch.likelihoods.BernoulliLikelihood()

### Build training loop

In [6]:
model.train()  # setting the state to training mode
likelihood.train()

BernoulliLikelihood()

In [7]:
optimizer = torch.optim.Adam(
    [{'params': model.parameters()},],  # Bernoulli Likelihood has no parameters
    lr=0.1,
)

In [8]:
# marginal log likelihood used for loss
mll = gpytorch.mlls.VariationalMarginalLogLikelihood(likelihood, model, num_data=len(train_y))

In [9]:
n = 50  # number of training iterations
pbar = tnrange(n)
for i in pbar:
    optimizer.zero_grad()
    output = model(train_x)
    loss = -1 * mll(output, train_y)  # negative log likelihood 
    loss.backward()  # backpropagation on the gradients
    
    print(f"{i+1:3d}/{n:3d} | Loss: {loss.item():4.2f}")
    
    optimizer.step()

  1/ 50 | Loss: 325.91
  2/ 50 | Loss: 229.14
  3/ 50 | Loss: 147.63
  4/ 50 | Loss: 94.62
  5/ 50 | Loss: 58.68
  6/ 50 | Loss: 34.10
  7/ 50 | Loss: 20.34
  8/ 50 | Loss: 14.26
  9/ 50 | Loss: 12.47
 10/ 50 | Loss: 11.96
 11/ 50 | Loss: 11.23
 12/ 50 | Loss: 10.48
 13/ 50 | Loss: 9.62
 14/ 50 | Loss: 8.82
 15/ 50 | Loss: 7.66
 16/ 50 | Loss: 6.76
 17/ 50 | Loss: 6.42
 18/ 50 | Loss: 5.32
 19/ 50 | Loss: 5.31
 20/ 50 | Loss: 5.46
 21/ 50 | Loss: 5.01
 22/ 50 | Loss: 4.91
 23/ 50 | Loss: 4.77
 24/ 50 | Loss: 4.99
 25/ 50 | Loss: 5.03
 26/ 50 | Loss: 4.72
 27/ 50 | Loss: 5.30
 28/ 50 | Loss: 4.87
 29/ 50 | Loss: 4.85
 30/ 50 | Loss: 4.88
 31/ 50 | Loss: 4.66
 32/ 50 | Loss: 4.53
 33/ 50 | Loss: 5.42
 34/ 50 | Loss: 4.82
 35/ 50 | Loss: 5.16
 36/ 50 | Loss: 4.89
 37/ 50 | Loss: 5.33
 38/ 50 | Loss: 4.72
 39/ 50 | Loss: 4.82
 40/ 50 | Loss: 4.38
 41/ 50 | Loss: 4.61
 42/ 50 | Loss: 4.80
 43/ 50 | Loss: 4.46
 44/ 50 | Loss: 4.82
 45/ 50 | Loss: 4.48
 46/ 50 | Loss: 4.99
 47/ 50 | Loss: 4.7

## Inference

In [10]:
model.eval()
likelihood.eval()

with torch.no_grad(), gpytorch.fast_pred_var():
    test_x = torch.linspace(0, 1, 101)
    observed_pred = likelihood(model(test_x))

### Plotting Predictions

In [11]:
f, ax = plt.subplots(1, 1, figsize=(12, 6), dpi=1024)
ax.plot(train_x.numpy(), train_y.numpy(), 'k*')
pred_labels = observed_pred.mean.ge(0.5).float().mul(2).sub(1) # ???
ax.plot(test_x.numpy(), pred_labels.numpy(), 'b')
ax.set_ylim([-3, 3])
ax.legend(['Observed Data', 'Mean', 'Confidence']);